# CVI: a tutorial


*Table of contents*
1. [Introduction](#Introduction)
2. [Model specification](#Model-specification)
3. [Limitations](#Limitations)
4. [Inference](#Inference)
5. [Extension](#Extension)

## Introduction

In this tutorial, we show how to excute VMP in models with delta factors of the form $\delta(f(x_{1}, \dots,  x_{n}) - y)$ with arbitrary differentiable function `f` inside with [Extended Variational Message Passing](https://www.mdpi.com/1099-4300/23/7/815) (EVMP) procedure.

More specifcly we will show:
1. [How specify](#Model-specification) a `Delta factor` of the form $\delta(f(x_{1}, \dots,  x_{n}) - y)$ inside the `@model` macro with EVMP as inference procedure
2. [What limitation](#Limitations) the current implementation has
3. [Show several inference examples](#Inference)
4. [How to extend](#Extension) it from the user perspective

## Model specification

Specifying an model is easy. Suppose we have a function `f`:

```
    f(x, y, ..., z) = ...
```

And we have a model where we want define a `Delta factor` with this `f`:

```
@model function model_name(...)
    ... some where here inputs (x, y, ..., z) defiened
    out ~ f(x, y, ..., z) where {meta = CVIApproximation(rng, n_iterations, n_samples, Descent(learning_rate))}
    ...    
end
```

If you want to see detalied example at this point go to [Inference](#Inference) section. 

The magic happens inside the `where` block: where we specifying trough `meta` parametr that `ReactiveMP` should run `EVMP` for the messages sended trough the `out ~ f(x, y, ..., z)` node.

So to specify it for `out ~ f(x, y, ..., z)` node you need set `meta` to `CVIApproximation(...)` inside `where` block: `meta=CVIApproximation(rng, n_samples, n_iterations, Descent(learning_rate))` in the above example.

The `CVIApproximation` structure serves for 2 reasons:
1. Marker that the `EVMP` rules need to be called
2. Container of the `EVMP` hyperparmetrs.

`EVMP` procedure has 4 hyperparametrs:
1. random number generator which will be called inside `EVMP` procedure (`rng`)
2. number of samples to use for the out message approximation (`n_samples`)
3. number of iteration of the EVMP procedure (`n_iterations`)
4. optimizer, which will be used to perform the EVMP step (`Descent(learning_rate)`)

## Limitations

There several main limitation for the `EVMP` procedure that you need satisfy:
1. The `EVMP` procedure suppose that there is mean-field assumpitoin on the interfaces connected to the node (`out, x, y, ..., z`)
2. The connected interface is factorized out in other nodes to which it connected
3. The messages on input interfaces (`x, y, ..., z`) are exponetial family distributions

In `ReactiveMP` you can obtain first and second assumption trough `@constraints` macro:

```
@model function model_name(...)
    ... some where here inputs (x, y, ..., z) defiened
    ... ~ Node1(x, q1, q2, ..., qn) # some node that is using x interface
    out ~ f(x, y, ..., z) where {meta = CVIApproximation(rng, n_iterations, n_samples, Descent(learning_rate))}
    ...
    ... ~ Node2(p1,..., out, pn) # some node that is using out interface
    ...    
end

constraints = @constraints begin
    q(out, p1, ..., pn) = q(y)q(p1,...,pn)
    q(out, x, y, ..., z) = q(out)q(x)...q(z)
    q(x, q1, ..., qn) = q(x)q(q1,...,qn)
end;
```

Note, that not all exponential family distributions are implemented. If you want to add one not implemented inside `ReactiveMP` you need to implement:
1. `naturalparams(dist)`. Function that returns a naturalparametrs from this distribution.
2. `lognormalizer(natparams)` lognormalizer for the naturalparametrs of this distribution


## Inference

## Extension